### Treehouse analysis stories coded up against the GA4GH Variant API

References:

https://ga4gh-schemas.readthedocs.io/en/latest/schemas/index.html

https://github.com/BD2KGenomics/bioapi-examples

https://github.com/ga4gh/ga4gh-client/blob/master/ga4gh/client/client.py

In [1]:
import pandas as pd
import numpy as np

!pip2 install -q --upgrade ga4gh --pre
import ga4gh
from ga4gh.client import client
print "ga4gh.client version", ga4gh.client.__version__

ga4gh.client version 0.0.4.dev9+ngd79dd00.d20170106


In [35]:
# Connect to the GA4GH server
c = client.HttpClient("http://1kgenomes.ga4gh.org")

# Print references in this server
print "References:\n", list(c.search_reference_sets())

# Use the first data set
datasets = list(c.search_datasets())
print "Data sets:\n", datasets
dataset_id = datasets[0].id

# Use the first variant set
variant_sets = list(c.search_variant_sets(dataset_id=dataset_id))
print "Variant sets:\n", [v.name for v in variant_sets]
variant_set_id=variant_sets[0].id

Data sets:
[id: "WyIxa2dlbm9tZXMiXQ"
name: "1kgenomes"
description: "Variants from the 1000 Genomes project and GENCODE genes annotations"
]
Variant sets:
[u'phase3-release', u'functional-annotation']
References:
[id: "WyJOQ0JJMzciXQ"
name: "NCBI37"
md5checksum: "54e0bb53844059bb7152618fc927cfa9"
ncbi_taxon_id: 9606
description: "NCBI37 assembly of the human genome"
assembly_id: "hg37"
source_uri: "ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/phase2_reference_assembly_sequence/hs37d5.fa.gz"
]


In [46]:
donor_id = "HG00356"

# call_sets = list(c.search_call_sets(variant_set_id=variant_sets[0].id, biosample_id=biosamples[0].id))
call_sets = list(c.search_call_sets(variant_set_id=variant_set_id, name=donor_id))
print "Call sets found:", len(call_sets)

Call sets found: 1


#### Variant Stories
As part of the overview, the analyst wants to review any of these variants present in the patient that are on this list: myCancerGenome.bed here: https://www.synapse.org/#!Synapse:syn6041633/wiki/407318. 

As part of the overview, the analyst wants to review  any high impact variants  present in the patient in cancer genes defined here: the Census_allWed Oct 5 22-03-15 2016.tsv list at those genes

We want to add the list of genes from these two searches to our enrichment analysis.

The variant coordinates included in the variant list are from GRCh37 and the gene coordinates in the gene list are from hg38. The variant input we want to match this info to is sometimes from bcbio analysis of exome or wgs, which we use grch37 for, and sometimes from variant calling on RNA-Seq, which uses hg38. The important aspect is the amino acid change, e.g. NRAS (Q61R), but it's easy got get it wrong when triangulating from different coordinates. In other words, the automation has to be validated variant by variant, and data source by data source.


https://www.synapse.org/#!Synapse:syn6041633/wiki/407318

Canonical variants to check for in every patient ("keep" list)
Obtained from Max H. on October 14, 2016
myCancerGenome.bed - coordinates from GRCh37

List of cancer genes to prioritize for variant review
Downloaded from Cancer Gene Census https://cancer.sanger.ac.uk/census by Olena on October 5, 2016:
Census_allWed Oct 5 22-03-15 2016.tsv -- coordinates from hg38

In [13]:
# Load high impact variants list (GRCh37)
high_impact_variants = pd.read_csv("treehouse/myCancerGenome.bed", sep="\t",
                                  names=["chrom", "start", "stop", "name", "details", "disease", "reference"])
high_impact_variants.head()

,chrom,start,stop,name,details,disease,reference
0,chr1,110881944,110881945,RBM15-MKL1 Fusion,RBM15-MKL1 Fusion,Acute Myeloid Leukemia,https://mycancergenome.org/content/disease/acu...
1,chr1,115247084,115247085,NRAS No Mutation Detected,NRAS No Mutation Detected,Colorectal Cancer,https://mycancergenome.org/content/disease/col...
2,chr1,115256528,115256529,Q61L Melanoma,NRAS c.182_183delAAinsTG (Q61L),Melanoma,https://mycancergenome.org/content/disease/mel...
3,chr1,115256528,115256529,Q61R Melanoma,NRAS c.182_183delAAinsGG (Q61R),Melanoma,https://mycancergenome.org/content/disease/mel...
4,chr1,115256528,115256529,Q61H Melanoma,NRAS c.183A>T (Q61H),Melanoma,https://mycancergenome.org/content/disease/mel...


In [38]:
%%time
# See if any of the high impact variants are present in the first call set
high_impact_calls = [(call, hv) for i, hv in high_impact_variants.iterrows() 
                     for call in c.search_variants(variant_set_id=variant_sets[0].id, call_set_ids=[call_sets[0].id],
                                                   reference_name=hv.chrom[3:], start=hv.start, end=hv.stop)
                     if call.start >= hv.start and call.end <= hv.stop]
print "High impact calls found:", len(high_impact_calls)

High impact calls found: 1
CPU times: user 1.06 s, sys: 56 ms, total: 1.12 s
Wall time: 4.78 s


As part of the overview, the analyst wants to review any high impact variants present in the patient in cancer genes defined here: the Census_allWed Oct 5 22-03-15 2016.tsv list at those genes

In [47]:
# Cancer genes
cancer_genes = pd.read_csv("treehouse/Census_allWed Oct  5 22-03-15 2016.tsv", sep="\t")
cancer_genes.head()

,Gene Symbol,Name,Entrez GeneId,Genome Location,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,ABI1,abl-interactor 1,10006,10:26748570-26860863,10p11.2,yes,NaN,AML,NaN,NaN,L,Dom,TSG,T,KMT2A,NaN,NaN,"ABI1,E3B1,ABI-1,SSH3BP1,10006"
1,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25,9:130835447-130885683,9q34.1,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,oncogene,"T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL1,p150,ABL,c-ABL,JTK7,bcr/abl,v-abl,P00519,..."
2,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27,1:179107718-179143044,1q24-q25,yes,NaN,AML,NaN,NaN,L,Dom,oncogene,T,ETV6,NaN,NaN,"ABL2,ARG,RP11-177A2_3,ABLL,P42684,ENSG00000143..."
3,ACKR3,atypical chemokine receptor 3,57007,2:-,2q37.3,yes,NaN,lipoma,NaN,NaN,M,Dom,oncogene,T,HMGA2,NaN,NaN,NaN
4,ACSL3,acyl-CoA synthetase long-chain family member 3,2181,2:222908773-222941654,2q36,yes,NaN,prostate,NaN,NaN,E,Dom,NaN,T,ETV1,NaN,NaN,"2181,PRO2194,ACS3,FACL3,O95573,ENSG00000123983..."


In [57]:
%%time
def calls_in_cancer_genes(cancer_genes, server, variant_set_id, call_set_id):
    for _, gene in cancer_genes.iterrows():
        reference_name, start, stop = re.match("(\d+)\:(\d+)-(\d+)", gene["Genome Location"]).groups()
        start = int(start)
        stop = int(stop)
        for call in server.search_variants(variant_set_id=variant_set_id, call_set_ids=[call_set_id],
                                           reference_name=reference_name, start=start, end=stop):
            if call.start >= start and call.end <= hv.stop:
                yield call
                
list(calls_in_cancer_genes(cancer_genes, c, variant_sets[0].id, call_sets[0].id))

AttributeError: 'NoneType' object has no attribute 'groups'